In [5]:
import pandas as pd
import mysql.connector as mysql
import re
import numpy as np

db = mysql.connect(
    host="localhost",
    user="root",
    password="secret",
    port="3306",
    database="serlo"
)

def flagged_word(word):
    flaggedWords = [
                    'online','official','call','fantasy','contact','installation', 
                    'pay', 'manufacturer', 'delivery', 'established', 'registration',
                    'seo', 'service', 'customers', 'trading', 'company', 'buy', 'sell', 
                    'escort', 'provider', 'product', 'patients', 'website', 'sites', 
                    'free', 'affordable', 'website', 'buisness', 'installed', 'install',
                    'management', 'manufacture', 'install', 'download', 'provides',
                    'downloaded', 'services', 'development', 'drug', 'top', 'reviews',
                    'review', 'treatment', 'deliver', 'hire', 'hiring', 'supply', 'supplier', 
                    'commercial', 'discount', 'account', 'websites', 'app', 'products', 
                    'guests', 'deals'
                    ]
    count = len(flaggedWords)
    counter = 0
    while counter <= count:
        if word.lower() == flaggedWords[counter]:
            return True
            break
        else: 
            counter = counter + 1
        if counter == count:
            return False


def unhelpful_word(word):
    unhelpfulWords = [
                      'col','plugin','rows','state','children','content','text',
                      'type','level','children','p','h','ttachment','ile','layout',
                      'child','image','src','width','alt','href'
                     ]
 
    count = len(unhelpfulWords)
    counter = 0
    while counter <= count:
        if word.lower() == unhelpfulWords[counter]:
            return True
            break
        else: 
            counter = counter + 1
        if counter == count:
            return False
        
        
def readable(splitString):
    resultData = ''
    for word in splitString:
        if word.isalpha():
            if unhelpful_word(word):
                pass
            else:
                resultData = resultData + ' ' + word
    return(resultData)


def description_length(description):
    descriptionLen = len(description)
    return (descriptionLen)

def query(sql):
    c = db.cursor()
    c.execute(sql)
    return c.fetchall()

potentialBots = np.empty((0,2))

database = query('''SELECT id, description FROM user WHERE description != "NULL"''' )

for user in database:
    flaggedWordCount = 0
    joinedString = "".join(user[1])
    splitString = re.split('"|/.| ', joinedString)
    descLen = description_length(splitString)
    
    for word in splitString:
        if flagged_word(word):
            flaggedWordCount = flaggedWordCount + 1
    if flaggedWordCount >= 1:
        potentialBots = np.vstack([potentialBots, [user[0], readable(splitString)]])
    elif descLen > 1500:
        potentialBots = np.vstack([potentialBots, user])
        
df = pd.DataFrame(potentialBots, columns = ['id', 'description'])
df.set_index('id', inplace=True)
print(df)


                                              description
id                                                       
1        As MSc Computer Science I enjoy forward think...
35408    multimedia explanation Hallo allerseits mein ...
68640    Shop online for prom bridesmaid wedding homec...
87383    Mathe English Infomatik Formel Deutsch buko n...
87518    was established in are an importer of fine le...
...                                                   ...
239628   Soft innovative solutions can help you achiev...
239631   My name is LaraJasmine I have been serving Ra...
239634                        a Buy Weed Online Australia
239720   a Bitcoin recovery online Well i just said le...
239777   hadir sebagai salah satu website a situs judi...

[103 rows x 1 columns]
